In [1]:
import photosom as ps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

# Load data from .dat file
train_raw = np.loadtxt('datasets/train_data.dat') #training region
test_raw = np.loadtxt('datasets/cv_data.dat') #sample region

# redshift, u, g, r, i, z, y
train = np.column_stack((train_raw[:,1], train_raw[:,4], train_raw[:,6], train_raw[:,8],
                         train_raw[:,10], train_raw[:,12], train_raw[:,14]))
test = np.column_stack((test_raw[:,1], test_raw[:,4], test_raw[:,6], test_raw[:,8],
                        test_raw[:,10], test_raw[:,12], test_raw[:,14]))

In [19]:
def _densityMap(index_map, title="", pixel=300, scheme='inferno'):
    densitymap = np.zeros((20, 20))
    for i in range(20):
        for j in range(20):
            if index_map[i,j] == None:
                densitymap[i,j] = 0
            else:
                densitymap[i,j] = len(index_map[i,j])
            
    x, y = np.meshgrid(range(0, 20), range(20-1,-1,-1))
    source = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'Galaxies': densitymap.ravel()})
    return alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', axis=alt.Axis(labels=False, title=None)),
        y=alt.Y('y:O', axis=alt.Axis(labels=False, title=None)),
        color=alt.Color("Galaxies:Q", scale=alt.Scale(scheme=scheme, type='linear'))
    ).properties(width=pixel,height=pixel,title=title)

def _colorMap(index_map, data, title="", pixel=300, scheme='purples', filter1=2, filter2=3):
    ri_colormap = np.zeros((20, 20))
    for i in range(20):
        for j in range(20):
            if index_map[i,j]:
                data_cut = data[index_map[i,j]]
                ri_color = data_cut[:,filter1] - data_cut[:,filter2]
                ri_colormap[i,j] = np.mean(ri_color)
                
    x, y = np.meshgrid(range(0, 20), range(20-1,-1,-1))
    source = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'Color': ri_colormap.ravel()})
    return alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', axis=alt.Axis(labels=False, title=None)),
        y=alt.Y('y:O', axis=alt.Axis(labels=False, title=None)),
        color=alt.Color("Color:Q", scale=alt.Scale(scheme=scheme, type='linear'))
    ).properties(width=pixel,height=pixel,title=title)

def _zMap(index_map, data, title="", pixel=300, scheme='viridis'):
    zmap = np.zeros((20, 20))
    for i in range(20):
        for j in range(20):
            if index_map[i,j]:
                data_cut = data[index_map[i,j]]
                zmap[i,j] = np.mean(data_cut)
                
    x, y = np.meshgrid(range(0, 20), range(20-1,-1,-1))
    source = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'Redshift': zmap.ravel()})
    return alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', axis=alt.Axis(labels=False, title=None)),
        y=alt.Y('y:O', axis=alt.Axis(labels=False, title=None)),
        color=alt.Color("Redshift:Q", scale=alt.Scale(scheme=scheme, type='linear'))
    ).properties(width=pixel,height=pixel,title=title)

def _zMeanErrorMap(index_map, data, quantile, title="", pixel=300, scheme='bluepurple', plot=True):
    z_err_map = np.zeros((20, 20))
    for i in range(20):
        for j in range(20):
            if index_map[i,j] != [[]]:
                data_cut = data[index_map[i,j]]
                z_err_map[i,j] = np.std(data_cut)/len(data_cut)
            else:
                z_err_map[i,j] = np.Inf
    
    cutoff = np.quantile(z_err_map[z_err_map!=np.Inf], quantile)
    for i in range(20):
        for j in range(20):
            if z_err_map[i,j] > cutoff:
                z_err_map[i,j] = np.nan
                
    x, y = np.meshgrid(range(0, 20), range(20-1,-1,-1))
    source = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'Deviation': z_err_map.ravel()})
    plt = alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', axis=alt.Axis(labels=False, title=None)),
        y=alt.Y('y:O', axis=alt.Axis(labels=False, title=None)),
        color=alt.Color("Deviation:Q", scale=alt.Scale(scheme=scheme, type='linear'))
    ).properties(width=pixel,height=pixel,title=title)
    if plot:
        return plt
    else:
        return z_err_map

def _cullMap(data, title="", pixel=300, scheme='greys'):
    cull_map = np.zeros((20, 20))
    for i in range(20):
        for j in range(20):
            if data[i,j]:
                cull_map[i,j] = 1
                
    x, y = np.meshgrid(range(0, 20), range(20-1,-1,-1))
    source = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'Culled': cull_map.ravel()})
    return alt.Chart(source).mark_rect().encode(
        x=alt.X('x:O', axis=alt.Axis(labels=False, title=None)),
        y=alt.Y('y:O', axis=alt.Axis(labels=False, title=None)),
        color=alt.Color("Culled:Q", scale=alt.Scale(scheme=scheme, type='linear'))
    ).properties(width=pixel,height=pixel,title=title)

<h2>First SOM without selection function</h2>

In [3]:
pz = ps.PhotoSOM(train, test, random_seed=338041)

z_cutoff_range = [0, 3.1]      # [0, 3.1]    for complete data
mag_cutoff_range = [14, 25.5]  # [14, 25.5]  for complete data
percentile_cut = 0.0           # 0.0         for complete data
n_bins = 100

pz.assignRange(z_cutoff_range, mag_cutoff_range, mag_filter=3)
pz.selectionFunction(30, percentile_cut, filter1=2, filter2=3)

48430 training and 9440 testing galaxies initialized


In [4]:
ml_pdfs = pz.randomForestTraining()
pz.predictionPlot(title="Predicted Redshfits vs. Truth Values")

There are 48430 training galaxies selected
No bins missing


alt.LayerChart(...)

<h2>Second SOM with selection function</h2>

In [5]:
pz2 = ps.PhotoSOM(train, test, random_seed=338041)

pz2.assignRange(z_cutoff_range, mag_cutoff_range, mag_filter=3)
pz2.selectionFunction(30, 0.5, filter1=2, filter2=3)

48430 training and 9440 testing galaxies initialized


In [6]:
ml_pdfs_2 = pz2.randomForestTraining()
pz2.predictionPlot(title="Predicted Redshfits vs. Truth Values")

There are 24262 training galaxies selected
4 bins missing, [1 2 3 6]


alt.LayerChart(...)

In [7]:
pz.initSOM(20)

data1 = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 1, pz.train_phot)
data2 = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 1, pz2.train_phot)
som = pz.som

In [8]:
map1 = som.win_map(data1, True)
map2 = som.win_map(data2, True)

index_map1 = np.empty((20, 20), dtype='O')
index_map2 = np.empty((20, 20), dtype='O')
for c in map1:
    if map1[c]:
        index_map1[c[1],c[0]] = map1[c]
    else:
        index_map1[c[1],c[0]] = [[]]
    if map2[c]:
        index_map2[c[1],c[0]] = map2[c]
    else:
        index_map2[c[1],c[0]] = [[]]

In [9]:
plot1 = _densityMap(index_map1)
plot2 = _densityMap(index_map2)

plot1 | plot2

alt.HConcatChart(...)

In [10]:
plot1 = _colorMap(index_map1, pz.train_phot)
plot2 = _colorMap(index_map2, pz2.train_phot)

plot1 | plot2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


alt.HConcatChart(...)

In [14]:
plot1 = _zMap(index_map1, pz.train_z)
plot2 = _zMap(index_map2, pz2.train_z)

plot1 | plot2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


alt.HConcatChart(...)

In [28]:
zmap = _zMeanErrorMap(index_map2, pz2.train_z, 0.65, plot=False)
cull_map = np.isnan(zmap)
print("Cull Rate: " + str(100*np.sum(cull_map)/(pz.size**2)) + "%")

_cullMap(cull_map)

Cull Rate: 40.25%


alt.Chart(...)